# Lets look at an example Linear Program for a supply constrainted problem 

**Example**

lets say that we are a manufacturing company and we have two main products (tables and chairs)

We can sell each `table for 30` and each `chair for 45`. But both tables and chairs are made up of similar materials and require a certain amount of each to produce one table or chair.

Say in order to make a table it requires `22 wood` and `15 steel`
and to make a chair it requires `45 wood` and `10 steel`.

But we only have a certain amount of `wood(5500)` and `steel(2500)` avalible to make our products. So we must distribute the resources effectively such that we waste the least amount and make the most amount of money.

So all in all 
    We want to Maximize total earned
    with constriants of
    total wood and steel avalible

**Converting Example to standard form**

we can reformulate the problem into a Maximization problem of we can make the amount of `tables = x` and amount of `chairs = x2`

so we want to find 

`f(x, x2) = Max(30x + 45x2)` with the constraints of wood `(22x + 45x2) <= 5500` and steel `(15x + 10x2) <= 2500` and of course we want to make a positive amount of tables and chairs so `x,x2 >= 0`. Concisely this looks like
```
f(x) = Max(30x + 45x2)
Subject to. 
(22x + 45x2) <= 5500
(15x + 10x2) <= 2500
x,x2 >= 0
```

*We can see this problem formulated and solved with the cvxpy library below, But if you were to solve it manualy look up simplex method since this is a linear program or look into the interior point method*

We can see a graphical solution (hence only two variable) https://www.desmos.com/calculator/su3zgtoa8j

In [23]:
import cvxpy as cp

# Defining the variables we want to solve for
x = cp.Variable()
x2 = cp.Variable()

# This is our objective function we want to maximize
objective = cp.Maximize(30*x + 45*x2)

# These are our constrints representive indiviually as inequalities
variableConstraints = (x >= 0)
variableConstraints1 = (x2 >= 0)
woodConstraint = (22*x + 45*x2 <= 5500)
steelConstraint = (15*x + 10*x2 <= 2500)
glueConstraint = (3*x + 2*x2 <= 550)
# we put all the constraints into a container(array) called constraints
constraints = [variableConstraints, variableConstraints1, woodConstraint, steelConstraint]

# Now we have our formulated problem
problem = cp.Problem(objective, constraints)

# outputing the results
print("The max amount of money we can make on these chairs and tables is:", problem.solve())
print("table (x) amount is: ", x.value)
print("chair (x2) amount is: ",  x2.value)

The max amount of money we can make on these chairs and tables is: 6510.989009341022
table (x) amount is:  126.37362623933787
chair (x2) amount is:  60.43956049246413


## Adding a new constraint (feasibility analysis) 

Lets go back to the example and say that we find out after computing our values that we need we find out that we actually have another product which we have a limited supply of and it is also required. That product is Glue

Say that for each `Table it costs 3 glue` and for a `Chair it cost 2 glue` and we only `have 550 glue`.

Then it would be pretty easy to add a new constraint as it would just be
```
glueConstraint = (3*x + 2*x2 <= 550)
```

now we can plug our previous values into the inequality and we can see if our previous anwser holds
```
3(126) + 2(60) <= 550 True
```
Now since the result was true then that means we dont need to do anything because we are already at a feasibility point. 


In [24]:
newConstraints = constraints + [3*x + 2*x2 <= 550]
problem = cp.Problem(objective, newConstraints)

print("The new max that we can possibly make:", problem.solve())
print("The new x and x2 values respectivly: ", x.value, x2.value)

The new max that we can possibly make: 6510.989009058838
The new x and x2 values respectivly:  126.37362628453253 60.439560456063575


We can see that the results were the same as without the new constraint so no need to recompute since we know this to be true.

But say instead we only have 300 glue avalible then that means 
```
3(126) + 2(60) <= 300 False
```
and then we would need to recompute a new feasible value

with the added constraint of 
```
glueConstraint = (3*x + 2*x2 <= 300)
```

In [25]:
newConstraints = constraints + [3*x + 2*x2 <= 300]
problem = cp.Problem(objective, newConstraints)

print("The new max that we can possibly make:", problem.solve())
print("The new x and x2 values respectivly: ", x.value, x2.value)

The new max that we can possibly make: 5719.7802170083605
The new x and x2 values respectivly:  27.472527201573207 108.79120891024809


We can see that we get drastically different values for x and x2 because of this new constraint. And we get a smaller value in the objective function

Now adding a new constraint is pretty straight forward but next we're going to take a look at adding a new product which invloves the dual problem.

## Adding new product (Dual Problem)